In [3]:
!mamba install ijson -c conda-forge -y

warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/
[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
conda-forge/noarch   ━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/linux-64   ━━━━━━━━━━━━━━━╸━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/noarch     ╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/linux-64      ╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s[+] 0.2s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━  76.2kB /  47.1MB @ 509.7kB/s  0.2s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━━━━━━━ 162.8kB /  22.4MB @   1.1MB/s  0.2s
pkgs/main/linux-64   ━━━━━━━━━━━━━━━━━━━━━━━ 122.3kB /   8.3MB @ 820.4kB/s  0.2s[+] 0.3s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━ 281.7kB /  47.1MB @   1.1MB/s

In [2]:
import os
import openslide
from histoplus.helpers.tissue_detection import detect_tissue_on_wsi

import json
import ijson
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image, ImageDraw, ImageFont
from typing import Dict, Tuple, List

import math
from collections import Counter
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator

In [3]:
wsi_path = '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-028_08.svs'
# wsi_path = '/home/sjhong/DB-000037-B2.svs'
os.path.exists(wsi_path)
slide = openslide.OpenSlide(wsi_path)
slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
tissue_coords, dz_level = detect_tissue_on_wsi(slide)
# Histoplus가 쓰는 tile_size=224, overlap=0 가정
dz = DeepZoomGenerator(slide, tile_size=224, overlap=0, limit_bounds=True)

TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
2025-09-23 11:48:46.791 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-09-23 11:48:46.793 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-09-23 11:48:52.904 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 19670 tiles with Otsu's threshold.


In [4]:
output_dir = './output/1_028_08/'
os.makedirs(output_dir, exist_ok = True)

In [5]:
TARGET_X = 225
TARGET_Y = 109
TARGET_LVL = 16

In [7]:
from draw_utils import get_base_patch, load_font, draw_annotation, draw_together

In [8]:
from IPython.display import display

In [9]:
font = load_font(14)
        
pad = 8


In [13]:
for i, INFERENCE_IMAGE_SIZE in enumerate(range(420,768,14)):
    # print(i)
    text = f"Inference image size: {INFERENCE_IMAGE_SIZE}"
    json_path = f"{output_dir}/seg_{wsi_path.split('/')[-1][:-4]}_{INFERENCE_IMAGE_SIZE}.json"

    with open(json_path, "r", encoding="utf-8") as f:
        results = json.load(f, parse_float=float)

    # 이제 바로 float로 접근 가능
    for t in results.get("cell_masks", []):
        if int(t.get("x", -1)) == TARGET_X and int(t.get("y", -1)) == TARGET_Y and int(t.get("level", -1)) == TARGET_LVL:
            # print("hit:", t)
            break    
    
    if t:
        tile_masks = t.get("masks", [])
        region_base, tile_info = get_base_patch(t, slide)
        canvas = draw_annotation(dz, tile_masks, region_base, tile_info)
        draw = draw_together(region_base, canvas)
        
        # 텍스트 추가
        d = ImageDraw.Draw(draw)
        bbox = d.textbbox((0,0), text, font=font)
        tw, th = bbox[2]-bbox[0], bbox[3]-bbox[1]
        
        # 위치: 오른쪽 아래 여백에
        x = draw.width - tw - pad*2
        y = draw.height - th - pad*2 - 10
        d.text((x+pad, y+pad), text,
            font=font, fill=(0, 0, 0))

        # display(draw)
        draw.save(f'/home/sjhong/segmentation/histoplus/tile_figure/infer_img_size/1_028_08_225_109_compared_{INFERENCE_IMAGE_SIZE}.png')
    else:
        print("not found")
    
    